In [42]:
from collections import deque, defaultdict
from tqdm import tqdm


def parse_map(racetrack):
    lines = racetrack.strip().split("\n")
    grid = [list(line) for line in lines]
    return grid


def find_positions(grid):
    start = None
    end = None
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            if val == "S":
                start = (r, c)
            elif val == "E":
                end = (r, c)
    return start, end


def bfs(grid, start, end, cheated_pos=None):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    visited = set([start])
    queue = deque([(start, 0)])  # (position, time)

    def is_walkable(x, y):
        within_bounds = 0 <= x < len(grid) and 0 <= y < len(grid[0])
        if not within_bounds:
            return False
        if (x, y) == end or grid[x][y] == ".":
            return True
        if grid[x][y] == "#" and cheated_pos and (x, y) in cheated_pos:
            return True
        return False

    while queue:
        (x, y), t = queue.popleft()

        if (x, y) == end:
            return t

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_walkable(nx, ny) and ((nx, ny) not in visited):
                visited.add((nx, ny))
                queue.append(((nx, ny), t + 1))

    return float("inf")


def find_cheats(grid):
    start, end = find_positions(grid)
    initial_time = bfs(grid, start, end)

    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    time_savings = defaultdict(int)
    used_walls = set()  # Track walls used in any cheat

    for x in tqdm(range(len(grid))):
        for y in range(len(grid[0])):
            if grid[x][y] == "#":
                continue

            for dx, dy in directions:
                for cheat_steps in range(1, 3):  # Cheat for 1 or 2 steps

                    cheated_positions = []
                    wall_positions = set()
                    for step in range(1, cheat_steps + 1):
                        cheat_x = x + dx * step
                        cheat_y = y + dy * step
                        if 0 <= cheat_x < len(grid) and 0 <= cheat_y < len(grid[0]):
                            cheated_positions.append((cheat_x, cheat_y))
                            if grid[cheat_x][cheat_y] == "#":
                                wall_positions.add((cheat_x, cheat_y))

                    if all(grid[cx][cy] == "#" for cx, cy in cheated_positions):
                        beyond_x = x + dx * (cheat_steps + 1)
                        beyond_y = y + dy * (cheat_steps + 1)
                        not_used_before = wall_positions.isdisjoint(used_walls)
                        if (
                            not_used_before
                            and 0 <= beyond_x < len(grid)
                            and 0 <= beyond_y < len(grid[0])
                            and (
                                grid[beyond_x][beyond_y] == "."
                                or (beyond_x, beyond_y) == end
                            )
                        ):

                            new_time = bfs(grid, start, end, set(cheated_positions))
                            saved_time = initial_time - new_time
                            if saved_time > 0:
                                time_savings[saved_time] += 1
                                used_walls.update(wall_positions)  # Add to used walls

    return time_savings


racetrack = """
###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
"""

grid = parse_map(racetrack)
cheats = find_cheats(grid)
for saving, count in sorted(cheats.items()):
    print(
        f"There {'is' if count == 1 else 'are'} {count} cheat{'s' if count > 1 else ''} that save {saving} picosecond{'s' if saving > 1 else ''}."
    )

100%|██████████| 15/15 [00:00<00:00, 2304.90it/s]

There are 14 cheats that save 2 picoseconds.
There are 14 cheats that save 4 picoseconds.
There are 2 cheats that save 6 picoseconds.
There are 4 cheats that save 8 picoseconds.
There are 2 cheats that save 10 picoseconds.
There are 3 cheats that save 12 picoseconds.
There is 1 cheat that save 20 picoseconds.
There is 1 cheat that save 36 picoseconds.
There is 1 cheat that save 38 picoseconds.
There is 1 cheat that save 40 picoseconds.
There is 1 cheat that save 64 picoseconds.


In [43]:
with open('./Data/Day 20/input.txt') as file:
    racetrack = file.read()


grid = parse_map(racetrack)
cheats = find_cheats(grid)
for saving, count in sorted(cheats.items()):
    print(
        f"There {'is' if count == 1 else 'are'} {count} cheat{'s' if count > 1 else ''} that save {saving} picosecond{'s' if saving > 1 else ''}."
    )

100%|██████████| 141/141 [01:14<00:00,  1.89it/s]

There are 979 cheats that save 2 picoseconds.
There are 979 cheats that save 4 picoseconds.
There are 293 cheats that save 6 picoseconds.
There are 459 cheats that save 8 picoseconds.
There are 198 cheats that save 10 picoseconds.
There are 321 cheats that save 12 picoseconds.
There are 143 cheats that save 14 picoseconds.
There are 230 cheats that save 16 picoseconds.
There are 102 cheats that save 18 picoseconds.
There are 182 cheats that save 20 picoseconds.
There are 80 cheats that save 22 picoseconds.
There are 143 cheats that save 24 picoseconds.
There are 58 cheats that save 26 picoseconds.
There are 108 cheats that save 28 picoseconds.
There are 52 cheats that save 30 picoseconds.
There are 111 cheats that save 32 picoseconds.
There are 56 cheats that save 34 picoseconds.
There are 103 cheats that save 36 picoseconds.
There are 42 cheats that save 38 picoseconds.
There are 77 cheats that save 40 picoseconds.
There are 33 cheats that save 42 picoseconds.
There are 73 cheats that

In [47]:
# Calculate and print total number of cheat instances saving more than 100 picoseconds
total_savings_over_100 = sum(count for saving, count in cheats.items() if saving >= 100)
print(
    f"\nTotal number of cheats that save more than 100 picoseconds: {total_savings_over_100}"
)


Total number of cheats that save more than 100 picoseconds: 1367


In [79]:
from collections import deque, defaultdict


def parse_map(racetrack):
    lines = racetrack.strip().split("\n")
    grid = [list(line) for line in lines]
    return grid


def find_positions(grid):
    start = None
    end = None
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            if val == "S":
                start = (r, c)
            elif val == "E":
                end = (r, c)
    return start, end


def bfs(grid, start, end, cheated_pos=None):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    visited = set([start])
    queue = deque([(start, 0)])  # (position, time)

    def is_walkable(x, y):
        within_bounds = 0 <= x < len(grid) and 0 <= y < len(grid[0])
        if not within_bounds:
            return False
        # Consider S and E as walkable for cheating purposes
        if grid[x][y] in (".", "S", "E"):
            return True
        if cheated_pos and (x, y) in cheated_pos:
            return True
        return False

    while queue:
        (x, y), t = queue.popleft()

        if (x, y) == end:
            return t

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_walkable(nx, ny) and ((nx, ny) not in visited):
                visited.add((nx, ny))
                queue.append(((nx, ny), t + 1))

    return float("inf")


def find_cheats(grid):
    start, end = find_positions(grid)
    initial_time = bfs(grid, start, end)

    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    unique_cheats = {}

    for x in range(len(grid)):
        for y in range(len(grid[0])):
            # Start can be '.', 'S', or 'E' (as endpoints can also start cheats)
            if grid[x][y] not in (".", "S", "E"):
                continue

            for dx, dy in directions:

                cheated_positions = []
                wall_positions = set()
                beyond_x, beyond_y = x, y
                hit_wall = False

                # Simulate the cheat and track the end position
                for step in range(1, 21 + 1):
                    cheat_x = x + dx * step
                    cheat_y = y + dy * step
                    if 0 <= cheat_x < len(grid) and 0 <= cheat_y < len(grid[0]):
                        if grid[cheat_x][cheat_y] == "#":
                            hit_wall = True
                            wall_positions.add((cheat_x, cheat_y))
                        beyond_x, beyond_y = cheat_x, cheat_y
                    cheated_positions.append((cheat_x, cheat_y))

                # Use frozenset for storing unique wall positions
                frozen_walls = frozenset(wall_positions)

                # Validate cheat end and uniqueness
                if hit_wall and frozen_walls not in unique_cheats.values():
                    # End can be '.', 'S', or 'E' (as endpoints can also end cheats)
                    if grid[beyond_x][beyond_y] in (".", "S", "E"):
                        cheat_key = (x, y, beyond_x, beyond_y)
                        if cheat_key not in unique_cheats:
                            new_time = bfs(grid, start, end, set(cheated_positions))
                            saved_time = initial_time - new_time
                            if saved_time >= 50:
                                unique_cheats[cheat_key] = frozen_walls

    time_savings = defaultdict(int)
    for saved_time in unique_cheats:
        _, _, ex, ey = saved_time
        cheat_walls = unique_cheats[saved_time]
        if (ex, ey) not in cheat_walls:  # Ensure the end lands correctly
            end_time = bfs(grid, start, end, cheat_walls)
            time_savings[initial_time - end_time] += 1

    return time_savings, unique_cheats


racetrack = """
###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
"""

grid = parse_map(racetrack)
cheats, unique_cheats = find_cheats(grid)

# Output the individual cheat information
for saving, count in sorted(cheats.items()):
    if saving >= 50:
        print(
            f"There {'is' if count == 1 else 'are'} {count} cheat{'s' if count > 1 else ''} that save {saving} picosecond{'s' if saving > 1 else ''}."
        )

In [89]:
from collections import deque, defaultdict


def parse_map(racetrack):
    lines = racetrack.strip().split("\n")
    grid = [list(line) for line in lines]
    return grid


def find_positions(grid):
    start = end = None
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            if val == "S":
                start = (r, c)
            elif val == "E":
                end = (r, c)
    return start, end


def bfs(grid, start, targets, prohibited=None):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    queue = deque([(start, 0)])
    visited = set([start])

    while queue:
        (x, y), t = queue.popleft()

        if (x, y) in targets:
            return t

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < len(grid)
                and 0 <= ny < len(grid[0])
                and (grid[nx][ny] in (".", "S", "E") or (nx, ny) in targets)
                and ((nx, ny) not in visited)
                and (prohibited is None or (nx, ny) not in prohibited)
            ):

                visited.add((nx, ny))
                queue.append(((nx, ny), t + 1))

    return float("inf")


def generate_rhombus(grid, x, y):
    max_distance = 20  # Maximum allowed cheat distance
    shortcuts = []

    # Nested loop to create a diamond shape
    for distance in range(1, max_distance + 1):
        for dx in range(-distance, distance + 1):
            dy = distance - abs(dx)
            for dy_try in [dy, -dy]:
                cx, cy = x + dx, y + dy_try
                if 0 <= cx < len(grid) and 0 <= cy < len(grid[0]):
                    if grid[cx][cy] == ".":
                        shortcuts.append((cx, cy))
                    elif grid[cx][cy] == "E":
                        shortcuts.append((cx, cy))
                    if grid[cx][cy] == "#":
                        break

    return shortcuts


def calculate_savings(grid):
    start, end = find_positions(grid)
    original_time = bfs(grid, start, {end})

    shortcut_savings = defaultdict(int)

    # Generate shortcuts and evaluate their time savings
    for x in range(len(grid)):
        for y in range(len(grid[0])):
            if grid[x][y] not in (".", "S", "E"):
                continue

            for ex, ey in generate_rhombus(grid, x, y):
                if (ex, ey) == (x, y):
                    continue

                sc_start = bfs(grid, start, {(x, y)})  # Path to shortcut start
                sc_end = bfs(grid, (ex, ey), {end})  # Path from shortcut end

                # Calculate the length of the shortcut (Manhattan distance)
                shortcut_length = abs(ex - x) + abs(ey - y)

                # Calculate the saved time
                if sc_start != float("inf") and sc_end != float("inf"):
                    total_time_with_shortcut = sc_start + sc_end + shortcut_length
                    saved_time = original_time - total_time_with_shortcut
                    if saved_time > 0:
                        key = ((x, y), (ex, ey))
                        shortcut_savings[key] = max(shortcut_savings[key], saved_time)

    return shortcut_savings


racetrack = """
###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
"""

grid = parse_map(racetrack)
shortcut_savings = calculate_savings(grid)

# Display results focusing on keys and their savings time
for ((sx, sy), (ex, ey)), saving in sorted(
    shortcut_savings.items(), key=lambda x: x[1], reverse=True
):
    if saving >= 50:
        print(f"Shortcut from ({sx}, {sy}) to ({ex}, {ey}) saves: {saving} picoseconds")

Shortcut from (3, 1) to (7, 3) saves: 76 picoseconds
Shortcut from (3, 1) to (7, 4) saves: 76 picoseconds
Shortcut from (3, 1) to (7, 5) saves: 76 picoseconds
Shortcut from (2, 1) to (7, 3) saves: 74 picoseconds
Shortcut from (2, 1) to (7, 4) saves: 74 picoseconds
Shortcut from (2, 1) to (7, 5) saves: 74 picoseconds
Shortcut from (3, 1) to (8, 3) saves: 74 picoseconds
Shortcut from (1, 1) to (7, 3) saves: 72 picoseconds
Shortcut from (1, 1) to (7, 4) saves: 72 picoseconds
Shortcut from (1, 1) to (7, 5) saves: 72 picoseconds
Shortcut from (1, 2) to (7, 3) saves: 72 picoseconds
Shortcut from (1, 2) to (7, 4) saves: 72 picoseconds
Shortcut from (1, 2) to (7, 5) saves: 72 picoseconds
Shortcut from (1, 3) to (7, 3) saves: 72 picoseconds
Shortcut from (1, 3) to (7, 4) saves: 72 picoseconds
Shortcut from (1, 3) to (7, 5) saves: 72 picoseconds
Shortcut from (2, 1) to (8, 3) saves: 72 picoseconds
Shortcut from (2, 3) to (7, 3) saves: 72 picoseconds
Shortcut from (2, 3) to (7, 4) saves: 72 picos

In [99]:
from collections import deque, defaultdict


def parse_map(racetrack):
    lines = racetrack.strip().split("\n")
    grid = [list(line) for line in lines]
    return grid


def find_positions(grid):
    start = end = None
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            if val == "S":
                start = (r, c)
            elif val == "E":
                end = (r, c)
    return start, end


def bfs(grid, start, targets, prohibited=None):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    queue = deque([(start, 0)])
    visited = set([start])

    while queue:
        (x, y), t = queue.popleft()

        if (x, y) in targets:
            return t

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < len(grid)
                and 0 <= ny < len(grid[0])
                and (grid[nx][ny] in (".", "S", "E") or (nx, ny) in targets)
                and ((nx, ny) not in visited)
                and (prohibited is None or (nx, ny) not in prohibited)
            ):

                visited.add((nx, ny))
                queue.append(((nx, ny), t + 1))

    return float("inf")


def generate_rhombus(grid, x, y):
    max_distance = 20  # Maximum allowed cheat distance
    shortcuts = []

    # Nested loop to create a diamond shape
    for distance in range(1, max_distance + 1):
        for dx in range(-distance, distance + 1):
            dy = distance - abs(dx)
            for dy_try in [dy, -dy]:
                cx, cy = x + dx, y + dy_try
                if 0 <= cx < len(grid) and 0 <= cy < len(grid[0]):
                    if grid[cx][cy] == ".":
                        shortcuts.append((cx, cy))
                    elif grid[cx][cy] == "E":
                        shortcuts.append((cx, cy))
                    if grid[cx][cy] == "#":
                        break

    return shortcuts


def calculate_savings(grid):
    start, end = find_positions(grid)
    original_time = bfs(grid, start, {end})

    shortcut_savings = defaultdict(int)

    # Generate shortcuts and evaluate their time savings
    for x in range(len(grid)):
        for y in range(len(grid[0])):
            if grid[x][y] not in (".", "S", "E"):
                continue

            for ex, ey in generate_rhombus(grid, x, y):
                if (ex, ey) == (x, y):
                    continue

                sc_start = bfs(grid, start, {(x, y)})  # Path to shortcut start
                sc_end = bfs(grid, (ex, ey), {end})  # Path from shortcut end

                # Calculate the length of the shortcut (Manhattan distance)
                shortcut_length = abs(ex - x) + abs(ey - y)

                # Calculate the saved time
                if sc_start != float("inf") and sc_end != float("inf"):
                    total_time_with_shortcut = sc_start + sc_end + shortcut_length
                    saved_time = original_time - total_time_with_shortcut
                    if saved_time > 0:
                        key = ((x, y), (ex, ey))
                        shortcut_savings[key] = max(shortcut_savings[key], saved_time)

    return shortcut_savings


def count_shortcuts_with_savings(shortcuts, min_savings):
    counts = defaultdict(int)
    for saving in shortcuts.values():
        if saving >= min_savings:
            counts[saving] += 1
    return counts


racetrack = """
###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
"""

grid = parse_map(racetrack)
shortcut_savings = calculate_savings(grid)

# Display results focusing on keys and their savings time
print("Shortcuts with significant savings:")
for ((sx, sy), (ex, ey)), saving in sorted(
    shortcut_savings.items(), key=lambda x: x[1], reverse=True
):
    if saving >= 50:
        print(f"Shortcut from ({sx}, {sy}) to ({ex}, {ey}) saves: {saving} picoseconds")

# Calculate and output the number of shortcuts saving a significant amount of time
min_savings = 50
shortcut_count = count_shortcuts_with_savings(shortcut_savings, min_savings)
print(f"Total number of shortcuts saving at least {min_savings} picoseconds:")
for saving, count in sorted(shortcut_count.items(), reverse=False):
    print(f"Savings: {saving} picoseconds, Count: {count}")

Shortcuts with significant savings:
Shortcut from (3, 1) to (7, 3) saves: 76 picoseconds
Shortcut from (3, 1) to (7, 4) saves: 76 picoseconds
Shortcut from (3, 1) to (7, 5) saves: 76 picoseconds
Shortcut from (2, 1) to (7, 3) saves: 74 picoseconds
Shortcut from (2, 1) to (7, 4) saves: 74 picoseconds
Shortcut from (2, 1) to (7, 5) saves: 74 picoseconds
Shortcut from (3, 1) to (8, 3) saves: 74 picoseconds
Shortcut from (1, 1) to (7, 3) saves: 72 picoseconds
Shortcut from (1, 1) to (7, 4) saves: 72 picoseconds
Shortcut from (1, 1) to (7, 5) saves: 72 picoseconds
Shortcut from (1, 2) to (7, 3) saves: 72 picoseconds
Shortcut from (1, 2) to (7, 4) saves: 72 picoseconds
Shortcut from (1, 2) to (7, 5) saves: 72 picoseconds
Shortcut from (1, 3) to (7, 3) saves: 72 picoseconds
Shortcut from (1, 3) to (7, 4) saves: 72 picoseconds
Shortcut from (1, 3) to (7, 5) saves: 72 picoseconds
Shortcut from (2, 1) to (8, 3) saves: 72 picoseconds
Shortcut from (2, 3) to (7, 3) saves: 72 picoseconds
Shortcut f

In [137]:
from collections import deque, defaultdict


def parse_map(racetrack):
    lines = racetrack.strip().split("\n")
    grid = [list(line) for line in lines]
    return grid


def bfs_from_point(grid, start, valid_chars):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    queue = deque([start])
    distances = {start: 0}

    while queue:
        x, y = queue.popleft()
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < len(grid)
                and 0 <= ny < len(grid[0])
                and grid[nx][ny] in valid_chars
                and (nx, ny) not in distances
            ):
                distances[(nx, ny)] = distances[(x, y)] + 1
                queue.append((nx, ny))

    return distances


def find_positions(grid):
    start = end = None
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            if val == "S":
                start = (r, c)
            elif val == "E":
                end = (r, c)
    return start, end


def find_possible_cheats(grid, start_to_coord):
    cheat_paths = defaultdict(dict)

    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    for sx, sy in start_to_coord:
        if grid[sx][sy] in (
            ".",
            "S",
            "E",
        ):  # Assuming cheats can start from any non-wall
            queue = deque([((sx, sy), 0)])  # (current position, current length)
            visited = set()

            while queue:
                (x, y), length = queue.popleft()

                if length >= 20:
                    continue  # Do not process if length exceeds max allowed

                for dx, dy in directions:
                    nx, ny = x + dx, y + dy
                    if (
                        0 <= nx < len(grid)
                        and 0 <= ny < len(grid[0])
                        and (nx, ny) not in visited
                    ):

                        # If it's a wall, consider passing through
                        if grid[nx][ny] == "#":
                            visited.add((nx, ny))
                            queue.append(((nx, ny), length + 1))

                        # If it's a valid endpoint ('.', 'S', 'E'), record the cheat
                        elif grid[nx][ny] in (".", "S", "E") and length > 0:
                            visited.add((nx, ny))
                            cheat_paths[(sx, sy)][(nx, ny)] = (
                                length + 1
                            )  # record the path length including this step

    return cheat_paths


def evaluate_cheats(grid, start, end, start_to_coord, coord_to_end, cheat_paths):
    original_time = start_to_coord[end]
    cheat_savings = defaultdict(int)

    for (sx, sy), exits in cheat_paths.items():
        for (ex, ey), cheat_length in exits.items():
            end_time = coord_to_end.get((ex, ey), float("inf"))
            if end_time != float("inf"):
                total_time_with_cheat = (
                    start_to_coord[(sx, sy)] + cheat_length + end_time
                )
                if total_time_with_cheat < original_time:
                    cheat_savings[((sx, sy), (ex, ey))] = max(
                        cheat_savings[((sx, sy), (ex, ey))],
                        original_time - total_time_with_cheat,
                    )

    return cheat_savings


def aggregate_savings(cheat_savings):
    savings_count = defaultdict(int)

    for saving in cheat_savings.values():
        # if saving >= 50:
        savings_count[saving] += 1

    return savings_count


# Sample racetrack
racetrack = """
###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############
"""

grid = parse_map(racetrack)
start, end = find_positions(grid)
start_to_coord = bfs_from_point(grid, start, {".", "E"})
coord_to_end = bfs_from_point(grid, end, {".", "S"})
cheat_paths = find_possible_cheats(grid, start_to_coord)
cheat_savings = evaluate_cheats(
    grid, start, end, start_to_coord, coord_to_end, cheat_paths
)
savings_count = aggregate_savings(cheat_savings)

print("Evaluated Cheat Paths and Savings:")
for ((sx, sy), (ex, ey)), saving in sorted(
    cheat_savings.items(), key=lambda x: x[1], reverse=True
):
    if saving >= 50:
        print(f"Cheat from ({sx}, {sy}) to ({ex}, {ey}) saves {saving} picoseconds.")

print("\nAggregated Savings:")
for saving, count in sorted(savings_count.items()):
    print(f"There are {count} cheats that save {saving} picoseconds")

Evaluated Cheat Paths and Savings:
Cheat from (3, 1) to (7, 3) saves 76 picoseconds.
Cheat from (3, 1) to (7, 4) saves 76 picoseconds.
Cheat from (3, 1) to (7, 5) saves 76 picoseconds.
Cheat from (3, 1) to (8, 3) saves 74 picoseconds.
Cheat from (2, 1) to (7, 3) saves 74 picoseconds.
Cheat from (2, 1) to (7, 4) saves 74 picoseconds.
Cheat from (2, 1) to (7, 5) saves 74 picoseconds.
Cheat from (3, 1) to (9, 1) saves 72 picoseconds.
Cheat from (3, 1) to (9, 2) saves 72 picoseconds.
Cheat from (2, 1) to (8, 3) saves 72 picoseconds.
Cheat from (1, 2) to (7, 3) saves 72 picoseconds.
Cheat from (1, 2) to (7, 4) saves 72 picoseconds.
Cheat from (1, 2) to (7, 5) saves 72 picoseconds.
Cheat from (3, 3) to (7, 3) saves 72 picoseconds.
Cheat from (3, 3) to (7, 4) saves 72 picoseconds.
Cheat from (3, 3) to (7, 5) saves 72 picoseconds.
Cheat from (3, 4) to (7, 4) saves 72 picoseconds.
Cheat from (3, 4) to (7, 5) saves 72 picoseconds.
Cheat from (3, 5) to (7, 5) saves 72 picoseconds.
Cheat from (2, 

In [138]:
racetrack_example = """
..E.
.##.
###.
S...
"""
example_grid = parse_map(racetrack_example)
start, end = find_positions(example_grid)
start_to_coord = bfs_from_point(example_grid, start, {".", "E"})
coord_to_end = bfs_from_point(example_grid, end, {".", "S"})
cheat_paths = find_possible_cheats(example_grid, start_to_coord)
cheat_savings = evaluate_cheats(
    example_grid, start, end, start_to_coord, coord_to_end, cheat_paths
)
savings_count = aggregate_savings(cheat_savings)


In [140]:
for saving, count in sorted(savings_count.items()):
    print(f"There are {count} cheats that save {saving} picoseconds")

There are 6 cheats that save 2 picoseconds


In [141]:
cheat_paths

defaultdict(dict,
            {(3, 0): {(3, 0): 2,
              (1, 0): 2,
              (3, 1): 3,
              (2, 3): 4,
              (3, 2): 4,
              (0, 1): 4,
              (1, 3): 5,
              (0, 2): 5},
             (3, 1): {(3, 1): 2,
              (2, 3): 3,
              (3, 2): 3,
              (3, 0): 3,
              (1, 0): 3,
              (0, 1): 3,
              (1, 3): 4,
              (0, 2): 4},
             (3, 2): {(2, 3): 2,
              (3, 2): 2,
              (3, 1): 3,
              (1, 3): 3,
              (0, 2): 3,
              (3, 0): 4,
              (1, 0): 4,
              (0, 1): 4},
             (2, 3): {(2, 3): 2,
              (3, 2): 2,
              (3, 1): 3,
              (1, 3): 3,
              (0, 2): 3,
              (3, 0): 4,
              (1, 0): 4,
              (0, 1): 4},
             (1, 3): {(1, 3): 2,
              (0, 2): 2,
              (2, 3): 3,
              (3, 2): 3,
              (1, 0): 3,
             